In [22]:
import requests
import pandas as pd
try:
    from bioinfokit import analys, visuz
except:
    !{sys.executable} -m pip install bioinfokit
    from bioinfokit import analys, visuz
from IPython.display import Image
import scipy
import os
from os import listdir

## Select file

In [304]:
path = "Mito/GSE116280/"

In [305]:
files = listdir(path)
files.sort()
print("The following files are located in the folder: \n"+str(files)) 

The following files are located in the folder: 
['GSE116280_Rot100nM_24h_8d.tsv', 'GSE116280_Rot50nM_24h_8d.tsv', 'cytoscape']


In [321]:
file = "GSE116280_Rot50nM_24h_8d.tsv"

In [322]:
df = pd.read_csv(path+file, sep='\t')
pd.set_option('display.float_format', '{:.6g}'.format)
df.head()

,ID,adj.P.Val,P.Value,t,B,logFC,REFSEQ,GENE_SYMBOL,GENE_NAME,ENSEMBL_ID
0,A_33_P3368771,0.00669,8.47e-07,-17.675,6.05276,-1.76653,NR_026991,H1FX-AS1,H1FX antisense RNA 1 (non-protein coding),ENST00000511998
1,A_23_P304897,0.00669,9.98e-07,17.2321,5.94032,1.81474,NM_000623,BDKRB2,bradykinin receptor B2,ENST00000554311
2,A_23_P170491,0.00669,1.14e-06,-16.8741,5.8457,-1.2453,NM_005879,TRAIP,TRAF interacting protein,ENST00000491060
3,A_23_P97700,0.00669,1.34e-06,-16.4556,5.73061,-2.23734,NM_006472,TXNIP,thioredoxin interacting protein,ENST00000475171
4,A_23_P339818,0.00669,1.54e-06,-16.1133,5.63277,-1.62187,NM_183376,ARRDC4,arrestin domain containing 4,ENST00000268042


In [323]:
visuz.gene_exp.volcano(d=df, lfc='logFC', pv='P.Value', lfc_thr = 0.2)
Image(url= "volcano.png", width=800, height=400)

In [324]:
print(len(df))

21729


Make new table with each row an entrez ID. Probes that mapped to multiple Entrez IDs will have multiple lines

In [325]:
df2 = pd.DataFrame(columns = ['GENE_SYMBOL','logFC','P.Value']) #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name
notincluded = []
for index, row in df.iterrows():
    try:
        for item in row['GENE_SYMBOL'].split('///'):  #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name
            df2.loc[len(df2.index)] = [item,row["logFC"],row["P.Value"]] 
    except:
        notincluded.append(row['ID'])
print('The following genes did not have gene IDs linked and will be excluded:')
print(notincluded)
print(len(notincluded))

The following genes did not have gene IDs linked and will be excluded:
['A_33_P3792489', 'A_32_P37495', 'A_24_P298587', 'A_33_P3294317', 'A_32_P133395', 'A_33_P3379076', 'A_33_P3549996', 'A_19_P00332068', 'A_33_P3249716', 'A_33_P3424872', 'A_33_P3416171', 'A_33_P3405680', 'A_33_P3282000', 'A_33_P3249837', 'A_33_P3298455', 'A_33_P3333995', 'A_33_P3408752', 'A_33_P3342752', 'A_33_P3273819', 'A_33_P3239594', 'A_24_P580698', 'A_33_P3339011', 'A_32_P62211', 'A_33_P3365924', 'A_33_P3215452', 'A_33_P3383044', 'A_33_P3262769', 'A_33_P3400319', 'A_33_P3422248', 'A_33_P3452287', 'A_32_P58699', 'A_24_P365679', 'A_33_P3402838', 'A_32_P88349', 'A_33_P3329448', 'A_33_P3398111', 'A_33_P3274627', 'A_33_P3265872', 'A_23_P354827', 'A_33_P3483909', 'A_24_P92680', 'A_33_P3236986', 'A_33_P3262449', 'A_33_P3288614', 'A_33_P3403723', 'A_32_P231493', 'A_33_P3331652', 'A_33_P3254761', 'A_23_P420942', 'A_33_P3287428', 'A_33_P3449815', 'A_33_P3260006', 'A_24_P187056', 'A_33_P3278103', 'A_33_P3302320', 'A_19_P003

In [326]:
df2.head()

,GENE_SYMBOL,logFC,P.Value
0,H1FX-AS1,-1.76653,8.47e-07
1,BDKRB2,1.81474,9.98e-07
2,TRAIP,-1.2453,1.14e-06
3,TXNIP,-2.23734,1.34e-06
4,ARRDC4,-1.62187,1.54e-06


In [327]:
df2.tail()

,GENE_SYMBOL,logFC,P.Value
21327,TRMT12,5.76e-05,1
21328,ADAM11,-2.97e-05,1
21329,HARBI1,1.1e-05,1
21330,GLRA3,4.18e-05,1
21331,LOC146795,-2.7e-06,1


In [328]:
len(df2)

21332

In [329]:
def Average(lst): 
    return sum(lst) / len(lst) 

In [330]:
a = {}
count = {}
for item in df2['GENE_SYMBOL']: #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name
    if not item in count:
        a[item]={}
        count[item]=1
        a[item]['P.Value']=[]
        a[item]['logFC']=[]
    else:
        count[item]+=1
print('Number of unique genes: '+str(len(count)))

Number of unique genes: 21257


In [331]:
for k,l in count.items():
    if l == 1:
        del a[k]
for index, row in df2.iterrows(): 
    if row['GENE_SYMBOL'] in a:  #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name
        a[row['GENE_SYMBOL']]['P.Value'].append(row['P.Value'])  #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name
        a[row['GENE_SYMBOL']]['logFC'].append(row['logFC'])  #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name
        df2.drop(index, inplace=True)
print(str(len(a)) + ' genes present multiple times in dataset.')
print(str(len(df2)) + ' genes present once in dataset.')

67 genes present multiple times in dataset.
21190 genes present once in dataset.


In [332]:
for item in a:
    x,y = scipy.stats.combine_pvalues(a[item]['P.Value'],method='fisher',weights=None)
    logfc = Average(a[item]['logFC'])
    df2 = df2.append({'GENE_SYMBOL':item,'logFC':logfc,'P.Value':y}, ignore_index=True)  #Gene.ID or GENE_SYMBOL  --> check dataset for ID column name

In [333]:
df2 = df2.sort_values(by=["P.Value"])

In [334]:
df2

,GENE_SYMBOL,logFC,P.Value
0,H1FX-AS1,-1.76653,8.47e-07
1,BDKRB2,1.81474,9.98e-07
2,TRAIP,-1.2453,1.14e-06
3,TXNIP,-2.23734,1.34e-06
4,ARRDC4,-1.62187,1.54e-06
5,PRSS56,1.70731,2.3e-06
6,DARC,-1.49763,2.35e-06
7,AMH,-1.74016,3.04e-06
8,C11orf96,1.22014,3.26e-06
9,C9orf24,-1.99847,3.63e-06


In [335]:
df2.to_csv(path+file[:-4]+"-processed.csv", index=False)